In [1]:
#This script is designed to do some quick formatting

In [2]:
import pandas as pd
import numpy as np
import re

In [ ]:
def seplatlon(df):
    print('in points') #this fr lat/lon
    df2=df[~df['lat'].str.contains("Points",na=False)].copy()#.to_csv('latlondatloc.csv',index=None)	
    df2=df2[df2['lat'].notnull()]
    df2['optrad'].fillna(0, inplace=True)
    df2['date'] = pd.to_datetime(df.date)
    df2=df2.drop_duplicates(subset=['lat','lon'], keep="first")#added for month visualization
    df2=df2.sort_values(by='date')
    df2.to_csv('latlondatloc.csv',index=None)

def sepbboxpoly(df):
    df3=df[df['lat'].str.contains("Points",na=False)].copy()
    df3.to_csv('latlonbbox.csv',index=None)

def sepcoordbboxpoly:
    df=pd.read_csv('latlondat.txt',dtype=str,encoding="ISO-8859-1")
    seplatlon(df)
    sepbboxpoly(df)

In [ ]:
sepcoordbboxpoly()

In [ ]:
#below is to format polygon and bounding box coordinates, while leaving all data intact

In [ ]:
pd.set_option('display.min_rows', 500)
df=pd.read_csv("latlonbbox.csv",usecols=["date","name","lat"])

In [ ]:
df.columns=["Date","Name","Points"]

In [ ]:
df['FormatLat'] = pd.Series(index=df.index)
df['IsBBOX'] = pd.Series(index=df.index)
df['LatVec'] = pd.Series(index=df.index)
df['LonVec'] = pd.Series(index=df.index)

In [ ]:
#pstr1=pstr.split(",")
#print(pstr1)
#re.split("(-{0,1}[0-9]*.[0-9]*-)",pstr)
def reampoint(stri):
    #print("stri= "+stri)
    c=stri.count("-")
    if c == 3 or (c==2 and not ("--" in stri)):
        #print("up")
        pos = stri.index('-', stri.index('-') + 1)
        strx = stri[:pos] + ',' + stri[pos + 1:]
    elif c==2 and ("--" in stri):
        #print("topmid")
        strx= stri.replace("-",",",1)
    elif c==1: #means there is none or second
        #print("down")
        #strx = stri[:pos] + ',' + stri[pos + 1:]
        strx=stri.replace("-",",")
    else: print("********\unknown format\n*******")
    #print("strx="+strx+"\n")
    
    return strx

for count,pstrall in enumerate(df["Points"]):#pstrall in df["Points"]:
    arr=[]
    pstrall=pstrall.replace("Points:","")
    pstrdat=pstrall.split(",")
    for stri in pstrdat:
        arr.append(reampoint(stri))
    if len(arr)>2:
        df['IsBBOX'][count]=False
    elif len(arr)<=2:
        df['IsBBOX'][count]=True
    strf='|'.join(arr)
    df['FormatLat'][count]=strf
#print(arr)

In [ ]:
dfbbox=df[df['IsBBOX']==True].reset_index()

In [ ]:
#strtest='(23.467122,120.247811)|(23.554839,120.345531)'
def bboxtovec(stri):
    theveclon=[]
    theveclat=[]
    afterarrs=[]
    arrs=stri.split('|')
    for stri1 in arrs:
        stri1=stri1.replace(")","")
        stri1=stri1.replace("(","")
        indiv=stri1.split(",")#indiv[0] is lat, indiv[1] is lon
        print(indiv)
        afterarrs.append(indiv)
        print('\n')
    print(afterarrs)
        #here we begin 4 point cases
    if afterarrs[0][0] < afterarrs[1][0] and afterarrs[0][1] < afterarrs[1][1]:
        print("in first")#good
        theveclon=[afterarrs[0][1],afterarrs[0][1],afterarrs[1][1],afterarrs[1][1]]
        theveclat=[afterarrs[0][0],afterarrs[1][0],afterarrs[1][0],afterarrs[0][0]]
    elif afterarrs[0][0] > afterarrs[1][0] and afterarrs[0][1] < afterarrs[1][1]:
        print("in second")#good
        theveclon=[afterarrs[0][1],afterarrs[1][1],afterarrs[1][1],afterarrs[0][1]]
        theveclat=[afterarrs[0][0],afterarrs[0][0],afterarrs[1][0],afterarrs[1][0]]
    elif afterarrs[0][0] > afterarrs[1][0] and afterarrs[0][1] > afterarrs[1][1]:
        print("in third")#good
        theveclon=[afterarrs[1][1],afterarrs[1][1],afterarrs[0][1],afterarrs[0][1]]
        theveclat=[afterarrs[1][0],afterarrs[0][0],afterarrs[0][0],afterarrs[1][0]]
    elif afterarrs[0][0] < afterarrs[1][0] and afterarrs[0][1] > afterarrs[1][1]:
        print("in fourth")#good
        theveclon=[afterarrs[1][1],afterarrs[0][1],afterarrs[0][1],afterarrs[1][1]]
        theveclat=[afterarrs[1][0],afterarrs[1][0],afterarrs[0][0],afterarrs[0][0]]    
    
    print("******")
    print(theveclon)
    print(theveclat)
    print("******")
    return [theveclon,theveclat]
    
#print(bboxtovec(strtest))

In [ ]:
for i in range(0,len(dfbbox.index)):
    retv=bboxtovec(dfbbox["FormatLat"][i])
    dfbbox["LatVec"][i]=retv[1]
    dfbbox["LonVec"][i]=retv[0]
    #print(df["FormatLat"][i])

In [ ]:
dfbbox2=dfbbox.drop_duplicates(subset=["FormatLat"],keep='first')

In [ ]:
dfbbox2.reset_index(inplace=True)

In [ ]:
dfbbox2

In [ ]:
st2=dfbbox2.to_csv(index=None).replace("[","").replace("]","").replace("'","")
with open("latlonbboxwvecreduced.csv","w") as f:
    f.write(st2)

In [ ]:
def polytovec(stri):
    theveclon=[]
    theveclat=[]
    afterarrs=[]
    arrs=stri.split('|')
    for stri1 in arrs:
        stri1=stri1.replace(")","")
        stri1=stri1.replace("(","")
        indiv=stri1.split(",")#indiv[0] is lat, indiv[1] is lon
        print(indiv)
        afterarrs.append(indiv)
        print('\n')
    print(afterarrs)
    
    for subarr in afterarrs:
        theveclat.append(subarr[0])
        theveclon.append(subarr[1])
    
    #print('theveclat=')
    #print(theveclat)
    #print('theveclon=')
    #print(theveclon)
    
    return [theveclon,theveclat]
    

In [ ]:
dfpoly=dfbbox=df[df['IsBBOX']==False].reset_index()

In [ ]:
dfpoly

In [ ]:
for i in range(0,len(dfpoly.index)):
    retv=polytovec(dfpoly["FormatLat"][i])
    dfpoly["LatVec"][i]=retv[1]
    dfpoly["LonVec"][i]=retv[0]

In [ ]:
dfpoly2=dfpoly.drop_duplicates(subset=["FormatLat"],keep='first')

In [ ]:
dfpoly2=dfpoly2.reset_index()

In [ ]:
st3=dfpoly2.to_csv(index=None).replace("[","").replace("]","").replace("'","")
with open("latlonpolywvecreduced.csv","w") as f:
    f.write(st3)